In [1]:
import pandas as pd
from metrics import *

In [2]:
df_my_bert = pd.read_csv('./submissions/my_kaggle_bert_submission.csv',index_col=0)
df_kaggle_bert = pd.read_csv('./submissions/roberta-base-unbiased-small_submission.csv',index_col=0)
df_roberta = pd.read_csv('./submissions/roberta-base-unbiased_submission.csv', index_col=0)

In [3]:
df_ensemble = df_my_bert

In [4]:
df_ensemble = df_ensemble.rename(columns={'my_kaggle_bert': 'my_ensemble'})

In [5]:
# all identities
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']
TOXICITY_COLUMN = 'target'
def convert_to_bool(df, col_name):
        df[col_name] = np.where(df[col_name] >= 0.5, True, False)
        
def convert_dataframe_to_bool(df):
    bool_df = df.copy()
    for col in ['target'] + identity_columns: # no target here
        convert_to_bool(bool_df, col)
    return bool_df
MODEL_NAME = 'my_ensemble'

In [132]:
df_ensemble = convert_dataframe_to_bool(df_ensemble)
for i in range(1, 9):
    j = 10 - i 

    df_ensemble['my_ensemble'] = df_my_bert['my_kaggle_bert']*(0.1*i)+ df_roberta['roberta-base-unbiased']* (0.1*j)
    bias_metrics_df = compute_bias_metrics_for_model(df_ensemble, identity_columns, MODEL_NAME, TOXICITY_COLUMN)
    bias_metrics_df.to_csv(f'./bias/{MODEL_NAME}_metrics.csv', index=False)
    print(f'{MODEL_NAME}, ({i},{j}): ', get_final_metric(bias_metrics_df, calculate_overall_auc(df_ensemble, MODEL_NAME)))
            

my_ensemble, (1,9):  0.9390518229855869
my_ensemble, (2,8):  0.9399730032118785
my_ensemble, (3,7):  0.9405413218732863
my_ensemble, (4,6):  0.9407657662686624
my_ensemble, (5,5):  0.940707910187318
my_ensemble, (6,4):  0.9403374762724831
my_ensemble, (7,3):  0.9396299820467724
my_ensemble, (8,2):  0.9386360005759561


In [133]:
df_ensemble = convert_dataframe_to_bool(df_ensemble)
for i in range(1, 9):
    for j in range(1, 9 - i):
        k = 10 - i - j
        df_ensemble['my_ensemble'] = df_my_bert['my_kaggle_bert']*(0.1*i) + df_kaggle_bert['roberta-base-unbiased-small']* (0.1*j) + df_roberta['roberta-base-unbiased']* (0.1*k)
        bias_metrics_df = compute_bias_metrics_for_model(df_ensemble, identity_columns, MODEL_NAME, TOXICITY_COLUMN)
        bias_metrics_df.to_csv(f'./bias/{MODEL_NAME}_metrics.csv', index=False)
        print(f'{MODEL_NAME}, ({i},{j},{k}): ', get_final_metric(bias_metrics_df, calculate_overall_auc(df_ensemble, MODEL_NAME)))
            

my_ensemble, (1,1,8):  0.9400689916329782
my_ensemble, (1,2,7):  0.9406993344621757
my_ensemble, (1,3,6):  0.940892637429088
my_ensemble, (1,4,5):  0.9408244918424135
my_ensemble, (1,5,4):  0.9405152641043976
my_ensemble, (1,6,3):  0.9399163932450181
my_ensemble, (1,7,2):  0.9389391390069313
my_ensemble, (2,1,7):  0.9408029898531265
my_ensemble, (2,2,6):  0.9412302928213221
my_ensemble, (2,3,5):  0.9413267934833698
my_ensemble, (2,4,4):  0.9411575338826912
my_ensemble, (2,5,3):  0.9406742633558489
my_ensemble, (2,6,2):  0.9398650571556274
my_ensemble, (3,1,6):  0.94120695794563
my_ensemble, (3,2,5):  0.9414774677546471
my_ensemble, (3,3,4):  0.9414415361901698
my_ensemble, (3,4,3):  0.941116534803025
my_ensemble, (3,5,2):  0.9404831787768949
my_ensemble, (4,1,5):  0.941294769866587
my_ensemble, (4,2,4):  0.9414082780731441
my_ensemble, (4,3,3):  0.941202690504928
my_ensemble, (4,4,2):  0.9407071110495272
my_ensemble, (5,1,4):  0.9410535309191365
my_ensemble, (5,2,3):  0.941001755360855

We pick `df_ensemble` = df_my_bert['my_kaggle_bert']* `0.6` + df_roberta['roberta-base-unbiased']* `0.4`

In [6]:
df_ensemble = convert_dataframe_to_bool(df_ensemble)
df_ensemble['my_ensemble'] = df_my_bert['my_kaggle_bert']*0.6+ df_roberta['roberta-base-unbiased']* 0.4
bias_metrics_df = compute_bias_metrics_for_model(df_ensemble, identity_columns, MODEL_NAME, TOXICITY_COLUMN)
bias_metrics_df.to_csv(f'./bias/{MODEL_NAME}_metrics.csv', index=False)
print(f'{MODEL_NAME}, ({0.6},{0.4}): ', get_final_metric(bias_metrics_df, calculate_overall_auc(df_ensemble, MODEL_NAME)))
df_ensemble.to_csv('./submissions/my_ensemble_submission.csv')

my_ensemble, (0.6,0.4):  0.9403374762724831
